In [ ]:
import cv2
from feat import Detector
import torch
import os
import pandas as pd

import logging, sys
logging.disable(sys.maxsize)

In [ ]:
detector = Detector(device='cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
emotion = {
    "angry": 0,
    "disgust": 1,
    "fear": 2,
    "happy": 3,
    "sad": 4,
    "surprise": 5,
    "neutral": 6
}

In [ ]:
class image_data:
    def __init__(self, path:str, real_emotion:str):
        self.path = path
        self.name = os.path.basename(path)
        self.image = cv2.imread(self.path)
        self.face = detector.detect_faces(self.image)
        self.landmark = detector.detect_landmarks(self.image, self.face)
#        self.emotion = detector.detect_emotions(self.image, self.face, self.landmark)
        self.au = detector.detect_aus(self.image, self.landmark)
#        self.processed = self.image
        self.real_emotion = real_emotion

In [ ]:
# Dataset 1
data = []
for em in emotion:
    path = os.path.join('./DiffusionFER/DiffusionEmotion_S/cropped/', em)

    files = os.listdir(path)
    files_abs = [os.path.abspath(os.path.join(path, file)) for file in files]
    data.append([image_data(name, em) for name in files_abs])
    print(f"Dataset {em} loaded!")

In [ ]:
# preparing the AUS data for storing
csv_file = []
csv_face = []
csv_emotion = []
csv_AUS = [
    [], [], [], [], [], [], [], [], [], [],
    [], [], [], [], [], [], [], [], [], []
]
au_numbers = [1, 2, 4, 5, 6, 7, 9, 10, 12, 14, 15, 17, 18, 20, 22, 23, 24, 25, 26, 27]

for em in data:
    for csv in em:
        for (face, aus) in enumerate(csv.au[0]):
            csv_file.append(csv.name)
            csv_emotion.append(csv.real_emotion)
            csv_face.append(face)
            for (i, au) in enumerate(aus):
                csv_AUS[i].append(au)

csv_data = {
    "file": csv_file,
    "emotion": csv_emotion,
    "face": csv_face,
}
aus_name = []
for (i, au_entry) in zip(au_numbers, csv_AUS):
    key = f"AUS{i:02d}"
    aus_name.append(key)
    csv_data[key] = au_entry

In [ ]:
# writing the csv file
df = pd.DataFrame(csv_data)
df.to_csv('aus.csv', index=False, header=True, sep=',', encoding='utf-8')